In [1]:
function only(itr)
    state = start(itr)
    val,state = next(itr, state)
    @assert(done(itr,state))
    return val
end

only (generic function with 1 method)

In [2]:
using SymEngine
using Expr2LaTeX
Expr2LaTeX.render(symexpr::SymEngine.Basic)=symexpr |> repr |> parse |> render

In [4]:
function weight_var(name, input_size, output_size)
    [symbols("$(name)_$j$i") for j in 1:output_size, i in 1:input_size]
end
function bias_var(name, output_size)
    [symbols("$(name)_$j") for j in 1:output_size]
end



bias_var (generic function with 1 method)

In [63]:
@vars x
W = weight_var("W", 1, 3)
b = bias_var("b",3)
V = weight_var("V", 3, 1)

z=(sigmoid.(W.^2*x + b))
N=V*z |> only

V_11/(1 + exp(-(b_1 + x*W_11^2))) + V_12/(1 + exp(-(b_2 + x*W_21^2))) + V_13/(1 + exp(-(b_3 + x*W_31^2)))

In [64]:
render(N)

L"$(\frac{V_{11}}{(1 + \mathrm{exp}(((b_{1} + (x * W_{11}^{2})))))} + \frac{V_{12}}{(1 + \mathrm{exp}(((b_{2} + (x * W_{21}^{2})))))} + \frac{V_{13}}{(1 + \mathrm{exp}(((b_{3} + (x * W_{31}^{2})))))})$"

In [65]:
render(diff(N,x))

L"$(\frac{(\mathrm{exp}(((b_{1} + (x * W_{11}^{2})))) * W_{11}^{2} * V_{11})}{(1 + \mathrm{exp}(((b_{1} + (x * W_{11}^{2})))))^{2}} + \frac{(\mathrm{exp}(((b_{2} + (x * W_{21}^{2})))) * W_{21}^{2} * V_{12})}{(1 + \mathrm{exp}(((b_{2} + (x * W_{21}^{2})))))^{2}} + \frac{(\mathrm{exp}(((b_{3} + (x * W_{31}^{2})))) * W_{31}^{2} * V_{13})}{(1 + \mathrm{exp}(((b_{3} + (x * W_{31}^{2})))))^{2}})$"